# Run TFlite inference on an example dataset

In [15]:
import json
import pandas as pd
import numpy as np
import tflite_runtime.interpreter as tflite

# Load inference arguments
with open("inference_args.json", "r") as f:
    inference_config = json.load(f)
selected_columns = inference_config["selected_columns"]

# Load captured landmarks
pq_file = "../output_landmarks.parquet"
df = pd.read_parquet(pq_file)

# Handle missing columns
missing_columns = [col for col in selected_columns if col not in df.columns]
for col in missing_columns:
    df[col] = 0.0  # Fill missing columns with zeros

# Ensure columns are in the correct order
frames = df[selected_columns].to_numpy(dtype=np.float32)

# Load TFLite model
model_path = "model.tflite"
interpreter = tflite.Interpreter(model_path=model_path)

# Verify input shape
input_details = interpreter.get_input_details()
print("Input Details:", input_details)

# Check input shape
expected_shape = input_details[0]['shape']  # [batch_size, num_features]
if len(frames.shape) != len(expected_shape):
    raise ValueError(f"Dimension mismatch: Expected {len(expected_shape)} dimensions, but got {len(frames.shape)}.")
if frames.shape[1] != expected_shape[1]:
    raise ValueError(f"Shape mismatch: Model expects {expected_shape[1]} features, but got {frames.shape[1]}.")

# Allocate tensors
interpreter.resize_tensor_input(input_details[0]['index'], frames.shape)
interpreter.allocate_tensors()

# Run inference
prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=frames)

# Decode predictions
with open("character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {v: k for k, v in character_map.items()}

# Get predicted string
prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output["outputs"], axis=1)])
print("Predicted Phrase:", prediction_str)


Input Details: [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([  1, 390], dtype=int32), 'shape_signature': array([ -1, 390], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Predicted Phrase: 2 a-e -aroe


In [8]:
import pandas as pd
pd.read_parquet("../output_landmarks.parquet").shape

(50, 1568)